[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [10]:
!pip install transformers

# Headers and Global Variables

In [11]:
from transformers import BertTokenizer
import csv

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')

SOURCE_FILE = '../datasets/cloze_test.csv'
TRIGGERS_ONLY_FILE = '../datasets/cloze_test_triggers_only.csv'
TRIGGERS_REMOVED_ONLY_FILE = '../datasets/cloze_test_triggers_removed_only.csv'
TRIGGERS_SYNONYMIZED_ONLY_FILE = '../datasets/cloze_test_triggers_synonymized_only.csv'

# Filters to remove tokens

In [12]:
def instead_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "instead":
            if i + 1 < len(tokens) and tokens[i + 1] == "of":
                pass
            else:
                tokens.pop(i)
                changed = True
                i = i - 1
        i = i + 1
    return changed

def ever_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "ever":
            if i > 0 and tokens[i - 1] == "than":
                pass
            else:
                tokens.pop(i)
                changed = True
                i = i - 1
        i = i + 1
    return changed

def anymore_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "anymore":
            tokens.pop(i)
            changed = True
            i = i - 1
        i = i + 1
    return changed

def too_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "too" and i + 1 < len(tokens) and tokens[i + 1] == ".":
            tokens.pop(i)
            i = i - 1
            if i >= 0 and tokens[i] == ",":
                tokens.pop(i)
                i = i - 1
            changed = True
        i = i + 1
    return changed

def eventually_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "eventually":
            tokens.pop(i)
            if i < len(tokens) and tokens[i] == ",":
                tokens.pop(i)
            i = i - 1
            changed = True
        i = i + 1
    return changed

def immediately_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "immediately":
            tokens.pop(i)
            i = i - 1
            changed = True
        i = i + 1
    return changed

def anyway_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "anyway":
            tokens.pop(i)
            i = i - 1
            changed = True
        i = i + 1
    return changed

def soon_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "soon":
            if i > 0 and tokens[i - 1] == "as":
                pass
            else:
                tokens.pop(i)
                if i < len(tokens) and tokens[i] == ",":
                    tokens.pop(i)
                i = i - 1
                changed = True
        i = i + 1
    return changed

def later_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "later":
            if i > 3 and tokens[i - 4] == "two" and tokens[i - 3] == "mile" and tokens[i - 2] == "##s" and tokens[i - 1] == "half":
                pass
            elif i > 1 and tokens[i - 2] == "month" and tokens[i - 1] == "##s":
                pass
            elif i > 1 and tokens[i - 2] == "week" and tokens[i - 1] == "##s":
                pass
            elif i > 1 and tokens[i - 2] == "day" and tokens[i - 1] == "##s":
                pass
            elif i > 0 and tokens[i - 1] == "year":
                pass
            else:
                tokens.pop(i)
                if i + 1 < len(tokens) and tokens[i] == "on" and tokens[i + 1] == ",":
                    tokens.pop(i)
                    tokens.pop(i)
                elif i < len(tokens) and tokens[i] == ",":
                    tokens.pop(i)
                elif i + 1 < len(tokens) and tokens[i] == "that" and tokens[i + 1] == "day":
                    tokens.pop(i)
                    tokens.pop(i)
                elif i + 2 < len(tokens) and tokens[i] == "in" and tokens[i + 1] == "his" and tokens[i + 2] == "life":
                    tokens.pop(i)
                    tokens.pop(i)
                    tokens.pop(i)
                i = i - 1
                changed = True
        i = i + 1
    return changed

def now_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "now":
            if i + 1 < len(tokens):
                if tokens[i + 1] == ",":
                    tokens.pop(i)
                    tokens.pop(i)
                    changed = True
                    i = i - 1
                elif tokens[i + 1] == ".":
                    tokens.pop(i)
                    changed = True
                    i = i - 1
        i = i + 1
    return changed

def finally_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "finally":
            tokens.pop(i)
            changed = True
            if i < len(tokens) and tokens[i] == ",":
                tokens.pop(i)
            i = i - 1
        i = i + 1
    return changed

filters = [instead_filter, ever_filter, anymore_filter, too_filter, eventually_filter, immediately_filter, anyway_filter, soon_filter, later_filter, now_filter, finally_filter]


# Filters to synonymize tokens

In [13]:
def instead_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "instead":
            if i + 1 < len(tokens) and tokens[i + 1] == "of":
                pass
            else:
                tokens[i] = "alternatively"
        i = i + 1

def ever_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "ever":
            if i > 0 and tokens[i - 1] == "than":
                pass
            else:
                tokens[i] = "at"
                tokens.insert(i + 1, "any")
                tokens.insert(i + 2, "point")
                i = i + 2
        i = i + 1

def anymore_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "anymore":
            tokens[i] = "any"
            tokens.insert(i + 1, "further")
            i = i + 1
        i = i + 1

def too_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "too" and i + 1 < len(tokens) and tokens[i + 1] == ".":
            tokens[i] = "as"
            tokens.insert(i + 1, "well")
            i = i + 1
        i = i + 1

def eventually_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "eventually":
            tokens[i] = "ultimately"
        i = i + 1

def immediately_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "immediately":
            tokens[i] = "instantly"
        i = i + 1

def anyway_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "anyway":
            tokens[i] = "any"
            tokens.insert(i + 1, "##how")
            i = i + 1
        i = i + 1

def soon_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "soon":
            if i > 0 and tokens[i - 1] == "as":
                pass
            else:
                tokens[i] = "shortly"
        i = i + 1

def later_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "later":
            if i > 3 and tokens[i - 4] == "two" and tokens[i - 3] == "mile" and tokens[i - 2] == "##s" and tokens[i - 1] == "half":
                pass
            elif i > 1 and tokens[i - 2] == "month" and tokens[i - 1] == "##s":
                pass
            elif i > 1 and tokens[i - 2] == "week" and tokens[i - 1] == "##s":
                pass
            elif i > 1 and tokens[i - 2] == "day" and tokens[i - 1] == "##s":
                pass
            elif i > 0 and tokens[i - 1] == "year":
                pass
            else:
                # no adequate synonyms, handled manually
                tokens[i] = "afterwards"
                if i + 1 < len(tokens) and tokens[i + 1] == "on" and tokens[i + 2] == ",":
                    tokens.pop(i + 1)
                    tokens.pop(i + 1)
        i = i + 1

def now_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "now":
            if i + 1 < len(tokens):
                tokens[i] = "currently"
        i = i + 1

def finally_synonymizer(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == "finally":
            tokens[i] = "at"
            tokens.insert(i + 1, "last")
            i = i + 1
        i = i + 1

synonymizers = [instead_synonymizer, ever_synonymizer, anymore_synonymizer, too_synonymizer, eventually_synonymizer, immediately_synonymizer, anyway_synonymizer, soon_synonymizer, later_synonymizer, now_synonymizer, finally_synonymizer]


# Execution

In [14]:
def execute(source_filename, triggers_removed_filename=None, triggers_synonymized_filename=None, triggers_only_filename=None, triggers_removed_only_filename=None, triggers_synonymized_only_filename=None):
    source_file = open(source_filename, 'r', encoding='utf-8')
    source_reader = csv.reader(source_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    triggers_removed_file = open(triggers_removed_filename, 'w', encoding='utf-8') if triggers_removed_filename is not None else None
    triggers_synonymized_file = open(triggers_synonymized_filename, 'w', encoding='utf-8') if triggers_synonymized_filename is not None else None
    triggers_only_file = open(triggers_only_filename, 'w', encoding='utf-8') if triggers_only_filename is not None else None
    triggers_removed_only_file = open(triggers_removed_only_filename, 'w', encoding='utf-8') if triggers_removed_only_filename is not None else None
    triggers_synonymized_only_file = open(triggers_synonymized_only_filename, 'w', encoding='utf-8') if triggers_synonymized_only_filename is not None else None

    triggers_removed_writer = csv.writer(triggers_removed_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True) if triggers_removed_filename is not None else None
    triggers_synonymized_writer = csv.writer(triggers_synonymized_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True) if triggers_synonymized_filename is not None else None
    triggers_only_writer = csv.writer(triggers_only_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True) if triggers_only_filename is not None else None
    triggers_removed_only_writer = csv.writer(triggers_removed_only_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True) if triggers_removed_only_filename is not None else None
    triggers_synonymized_only_writer = csv.writer(triggers_synonymized_only_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True) if triggers_synonymized_only_filename is not None else None

    source_lines = []

    for source_line in source_reader:
        source_lines.append(source_line)

    triggers_removed_writer.writerow(source_lines[0]) if triggers_removed_filename is not None else None
    triggers_synonymized_writer.writerow(source_lines[0]) if triggers_synonymized_filename is not None else None
    triggers_only_writer.writerow(source_lines[0]) if triggers_only_filename is not None else None
    triggers_removed_only_writer.writerow(source_lines[0]) if triggers_removed_only_filename is not None else None
    triggers_synonymized_only_writer.writerow(source_lines[0]) if triggers_synonymized_only_filename is not None else None
    _ = source_lines.pop(0)

    for source_line in source_lines:
        changed = False
        tokens1_to_remove = tokenizer.tokenize(source_line[-3])
        tokens1_to_synonymize = tokens1_to_remove.copy()
        tokens2_to_remove = tokenizer.tokenize(source_line[-2])
        tokens2_to_synonymize = tokens2_to_remove.copy()

        for filter, synonymizer in zip(filters, synonymizers):
            changed = changed or filter(tokens1_to_remove) or filter(tokens2_to_remove)
            synonymizer(tokens1_to_synonymize)
            synonymizer(tokens2_to_synonymize)

        sentence1_triggers_removed = tokenizer.convert_tokens_to_string(tokens1_to_remove)
        sentence2_triggers_removed = tokenizer.convert_tokens_to_string(tokens2_to_remove)
        triggers_removed_line = source_line[:-3] + [sentence1_triggers_removed] + [sentence2_triggers_removed] + source_line[-1:]
        triggers_removed_writer.writerow(triggers_removed_line) if triggers_removed_filename is not None else None

        sentence1_triggers_synonymized = tokenizer.convert_tokens_to_string(tokens1_to_synonymize)
        sentence2_triggers_synonymized = tokenizer.convert_tokens_to_string(tokens2_to_synonymize)
        triggers_synonymized_line = source_line[:-3] + [sentence1_triggers_synonymized] + [sentence2_triggers_synonymized] + source_line[-1:]
        triggers_synonymized_writer.writerow(triggers_synonymized_line) if triggers_synonymized_filename is not None else None

        if changed:
            triggers_only_writer.writerow(source_line) if triggers_only_filename is not None else None
            triggers_removed_only_writer.writerow(triggers_removed_line) if triggers_removed_only_filename is not None else None
            triggers_synonymized_only_writer.writerow(triggers_synonymized_line) if triggers_synonymized_only_filename is not None else None

    source_file.close()
    triggers_removed_file.close() if triggers_removed_filename is not None else None
    triggers_synonymized_file.close() if triggers_synonymized_filename is not None else None
    triggers_only_file.close() if triggers_only_filename is not None else None
    triggers_removed_only_file.close() if triggers_removed_only_filename is not None else None
    triggers_synonymized_only_file.close() if triggers_synonymized_only_filename is not None else None

In [15]:
if __name__ == "__main__":
    execute(SOURCE_FILE, triggers_only_filename=TRIGGERS_ONLY_FILE, triggers_removed_only_filename=TRIGGERS_REMOVED_ONLY_FILE, triggers_synonymized_only_filename=TRIGGERS_SYNONYMIZED_ONLY_FILE)